# Download videos from the emplusvault API

This notebook downloads videos from the API and saves the metadata to a CSV file.

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from getpass import getpass
import requests
from tqdm.notebook import tqdm
import urllib3
import json
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [36]:
username = input("Enter your username: ")
password = getpass("Enter your password: ")

In [37]:
username

'vita'

In [38]:
BASE_URL = "https://10.179.68.6:32034"

data = {
    "grant_type": "password",
    "username": username,
    "password": password,
}

In [5]:
response = requests.post(f"{BASE_URL}/gettoken", data=data, verify=False)

if response.status_code != 200:
    print("Authentication failed!")

json_response = response.json()
access_token = json_response["access_token"]

headers = {
    "Authorization": f"Bearer {access_token}"
}

/Users/arattinger/Projects/rts/rts/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.179.68.6'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
library_id = 2
last_seen_value = 0
page_size = 100
response = requests.get(f"{BASE_URL}/media/library/{library_id}", params={"last_seen_value": last_seen_value, "page_size": page_size}, headers=headers, verify=False)


/Users/arattinger/Projects/rts/rts/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.179.68.6'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [13]:
results = []

library_id = 2
last_seen_value = 0
page_size = 100
response = requests.get(f"{BASE_URL}/media/library/{library_id}", params={"last_seen_value": last_seen_value, "page_size": page_size}, headers=headers, verify=False)
results.extend(response.json())

while c:= requests.get(f"{BASE_URL}/media/library/{library_id}", params={"last_seen_date": results[-1]['created_at'], "last_seen_media_id":  results[-1]['media_id'], "page_size": page_size}, headers=headers, verify=False).json():
    results.extend(c)
    # print(len(c))

In [39]:
len(results)

154231

In [75]:
with open("IOC_metadata.json", "w") as f:
    json.dump(results, f)

In [40]:
import os

In [41]:
def authenticate():
    response = requests.post(f"{BASE_URL}/gettoken", data=data, verify=False)
    json_response = response.json()
    access_token = json_response["access_token"]

    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    return headers

def download_video(media_id: str):

    headers = authenticate()
    fn = f"/Volumes/EMP8TB/videos/{media_id}.mp4"
    try:
        file_stats = os.stat(fn)

        if file_stats.st_size > 10000:
            return
    except FileNotFoundError:
        pass

    response = requests.get(f"{BASE_URL}/download/{media_id}", headers=headers, verify=False)
    with open(fn, "wb") as f:
        # print(media_id, response.status_code)
        if response.status_code != 200:
            headers = authenticate()
            # raise ValueError(f"Error downloading {media_id}, status code {response.status_code}, {response.content}")
        f.write(response.content)

In [43]:
for r in tqdm(results[10000:]):
    download_video(r['media_id'])

  0%|          | 0/144231 [00:00<?, ?it/s]